In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


### 1.**先读取数据**

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/kagglex/train.csv')
test= pd.read_csv('/content/drive/MyDrive/kagglex/test.csv')

In [ ]:
#简略查看训练数据
#很明显，测试集里面的id是没有用的，要先去除掉
#train = train.drop(columns = ['id'])
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [ ]:
train.shape

(188533, 13)

In [ ]:
test_id = test['id']
print(test_id.head())
test.shape

0    188533
1    188534
2    188535
3    188536
4    188537
Name: id, dtype: int64


(125690, 12)

In [ ]:
test.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [ ]:
#test = test.drop(columns = ['id'])

## **2.进行数据分析**

### **2.1 构建、拆分新变量**

In [ ]:
# 先构建新的变量 车龄
train['car_age'] = 2024 - train['model_year']
test['car_age'] = 2024 - test['model_year']
train = train.drop(columns=['model_year'])
test = test.drop(columns=['model_year'])

In [ ]:
# 构建新变量 公里
train['kmilage'] = train['milage']/1000
test['kmilage'] = test['milage']/1000
train = train.drop(columns=['milage'])
test = test.drop(columns=['milage'])

In [ ]:
# 然后我想拆分出新的特征变量 根据engine这一项 拆分出horsepower和engine_displacement
train['horsepower'] = train['engine'].str.extract(r'(\d+\.?\d*)HP', expand=False)
test['horsepower'] = test['engine'].str.extract(r'(\d+\.?\d*)HP', expand=False)

train['engine_displacement'] = train['engine'].str.extract(r'(\d+\.\d+)(?:L| Liter| Liter Turbo| Liter GTDI| Liter DOHC Twin Turbo| Liter SC ULEV)', expand=False)
test['engine_displacement'] = test['engine'].str.extract(r'(\d+\.\d+)(?:L| Liter| Liter Turbo| Liter GTDI| Liter DOHC Twin Turbo| Liter SC ULEV)', expand=False)

# 将提取到的值转为浮点数，并处理空值
train['horsepower'] = train['horsepower'].replace('', np.nan).astype(float)
train['engine_displacement'] = train['engine_displacement'].replace('', np.nan).astype(float)

test['horsepower'] = test['horsepower'].replace('', np.nan).astype(float)
test['engine_displacement'] = test['engine_displacement'].replace('', np.nan).astype(float)

#丢弃原来变量
train = train.drop(columns=['engine'])
test = test.drop(columns=['engine'])

In [ ]:
#clean_title 没有用
train = train.drop(columns='clean_title')
test = test.drop(columns='clean_title')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   188533 non-null  int64  
 1   brand                188533 non-null  object 
 2   model                188533 non-null  object 
 3   fuel_type            183450 non-null  object 
 4   transmission         188533 non-null  object 
 5   ext_col              188533 non-null  object 
 6   int_col              188533 non-null  object 
 7   accident             186081 non-null  object 
 8   price                188533 non-null  int64  
 9   car_age              188533 non-null  int64  
 10  kmilage              188533 non-null  float64
 11  horsepower           155274 non-null  float64
 12  engine_displacement  181763 non-null  float64
dtypes: float64(3), int64(3), object(7)
memory usage: 18.7+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   125690 non-null  int64  
 1   brand                125690 non-null  object 
 2   model                125690 non-null  object 
 3   fuel_type            122307 non-null  object 
 4   transmission         125690 non-null  object 
 5   ext_col              125690 non-null  object 
 6   int_col              125690 non-null  object 
 7   accident             124058 non-null  object 
 8   car_age              125690 non-null  int64  
 9   kmilage              125690 non-null  float64
 10  horsepower           103509 non-null  float64
 11  engine_displacement  121252 non-null  float64
dtypes: float64(3), int64(2), object(7)
memory usage: 11.5+ MB


### **2.2 合并、观察数据**

In [ ]:
#分成两个类型，数值属性和非数值属性
train_numeric_features = ['car_age','kmilage','horsepower','engine_displacement','price']
test_numeric_features = ['car_age','kmilage','horsepower','engine_displacement']
category_features = ['brand','model','fuel_type','transmission','ext_col','int_col','accident']

In [ ]:
# 合并到一起，看每个变量有多少唯一值 然后再确定如何把object类型转换
all_data = pd.concat([train,test],axis=0,ignore_index=True)
all_data.head()

,id,brand,model,fuel_type,transmission,ext_col,int_col,accident,price,car_age,kmilage,horsepower,engine_displacement
0,0,MINI,Cooper S Base,Gasoline,A/T,Yellow,Gray,None reported,4200.0,17,213.000,172.0,1.6
1,1,Lincoln,LS V8,Gasoline,A/T,Silver,Beige,At least 1 accident or damage reported,4999.0,22,143.250,252.0,3.9
2,2,Chevrolet,Silverado 2500 LT,E85 Flex Fuel,A/T,Blue,Gray,None reported,13900.0,22,136.731,320.0,5.3
3,3,Genesis,G90 5.0 Ultimate,Gasoline,Transmission w/Dual Shift Mode,Black,Black,None reported,45000.0,7,19.500,420.0,5.0
4,4,Mercedes-Benz,Metris Base,Gasoline,7-Speed A/T,Black,Beige,None reported,97500.0,3,7.388,208.0,2.0


In [ ]:
#先通过info()看看所有数据的数据类型
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314223 entries, 0 to 314222
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   314223 non-null  int64  
 1   brand                314223 non-null  object 
 2   model                314223 non-null  object 
 3   fuel_type            305757 non-null  object 
 4   transmission         314223 non-null  object 
 5   ext_col              314223 non-null  object 
 6   int_col              314223 non-null  object 
 7   accident             310139 non-null  object 
 8   price                188533 non-null  float64
 9   car_age              314223 non-null  int64  
 10  kmilage              314223 non-null  float64
 11  horsepower           258783 non-null  float64
 12  engine_displacement  303015 non-null  float64
dtypes: float64(4), int64(2), object(7)
memory usage: 31.2+ MB


In [ ]:
# 首先我想知道每个非数字特征有多少独特值
for feature in category_features:
  print(feature+"的特征分布如下：")
  print("{}特征有{}个不同的值".format(feature,all_data[feature].nunique()))
  # print(all_data[feature].value_counts())
  # print(72 * '-')

brand的特征分布如下：
brand特征有57个不同的值
model的特征分布如下：
model特征有1898个不同的值
fuel_type的特征分布如下：
fuel_type特征有7个不同的值
transmission的特征分布如下：
transmission特征有52个不同的值
ext_col的特征分布如下：
ext_col特征有319个不同的值
int_col的特征分布如下：
int_col特征有156个不同的值
accident的特征分布如下：
accident特征有2个不同的值


### **2.3 存在空缺值 要先对空缺值进行处理**

In [ ]:
#这个‘-'空缺值是我观察数据集得来的
#如果只针对空值做处理的话，这种异常值就很容易漏掉
#未来别的也需要注意这一点
all_data.replace('–', np.nan, inplace=True)

In [ ]:
all_data.isnull().sum()

,0
id,0
brand,0
model,0
fuel_type,9785
transmission,127
ext_col,589
int_col,7480
accident,4084
price,125690
car_age,0


#### **2.3.1 transmission特征处理**

In [ ]:
#由于127个数没有值，我们就直接用众数来填充，我觉得就OK？
all_data['transmission'].replace(np.nan, all_data['transmission'].mode()[0], inplace=True)

#### **2.3.2 accident特征处理**

In [ ]:
#accident特征只有两个变量 根据比例来随机填充空缺值
missing_indices = all_data[all_data['accident'].isnull()].index

# 计算比例
none_reported_ratio = 240777 / (240777 + 69362)
accident_reported_ratio = 69362 / (240777 + 69362)

# 根据比例随机选择填充
# np.random.choice 会根据给定的概率随机选择 0 或 1 进行填充
random_fill = np.random.choice(['None reported', 'At least 1 accident or damage reported'], size=len(missing_indices), p=[none_reported_ratio, accident_reported_ratio])

# 用随机生成的值填充缺失值
all_data.loc[missing_indices, 'accident'] = random_fill

# 查看填充结果
print(all_data['accident'].value_counts())


accident
None reported                             244001
At least 1 accident or damage reported     70222
Name: count, dtype: int64


In [ ]:
mapping = {
    'None reported': 0,
    'At least 1 accident or damage reported': 1
}

all_data['accident'] = all_data['accident'].map(mapping)

#### **2.3.3 fuel_type特征处理**

In [ ]:
# 通过一个简单的随机森林模型进行填充，
# 因为很多变量还是空白的，并且有些变量也没转变成数字型，就随便拟合就好我觉得
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 特征与目标变量
features = ['car_age', 'kmilage','accident']  # 假设这些是相关特征
target = 'fuel_type'

# 去除缺失值的样本
df_non_missing = all_data.dropna(subset=[target])
X = df_non_missing[features]
y = df_non_missing[target]

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练模型
model = RandomForestClassifier(n_estimators=200,min_samples_leaf=2)
model.fit(X_train, y_train)

# 测试模型
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

# 预测缺失值
df_missing = all_data[all_data[target].isnull()]
X_missing = df_missing[features]
predicted_fuel_type = model.predict(X_missing)

# # 填充缺失值
all_data.loc[all_data[target].isnull(), target] = predicted_fuel_type


Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters found: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 20, 'bootstrap': True}
Improved Accuracy: 0.910907789004694


ValueError: Found array with 0 sample(s) (shape=(0, 3)) while a minimum of 1 is required by RandomForestClassifier.

#### **2.3.4 ext_col和int_col特征处理**

In [ ]:
from collections import Counter
import pandas as pd
import numpy as np

# 找到 ext_col 特征中值为 np.nan 的行
dash_ext_col_rows = all_data[pd.isna(all_data['ext_col'])]

# 找到 int_col 特征中值为 np.nan 的行
dash_int_col_rows = all_data[pd.isna(all_data['int_col'])]

# 定义一个函数来填充 col 特征中的 np.nan
def fill_col(row, color):
    # 找到与当前行品牌相同且 car_age 在前后三年内的所有行
    same_brand_rows = all_data[(all_data['brand'] == row['brand']) &
                               (all_data['car_age'] >= row['car_age'] - 1) &
                               (all_data['car_age'] <= row['car_age'] + 1) &
                               pd.notna(all_data[color])]

    # 如果找不到满足条件的行，则返回原值
    if same_brand_rows.empty:
        return row[color]

    # 找到这些行中 col 出现最多的值
    most_common_col = Counter(same_brand_rows[color]).most_common(1)[0][0]

    # 返回出现最多的 col 值
    return most_common_col

# 将特征值为 np.nan 的行的 ext_col 和 int_col 特征值替换
all_data['ext_col'] = all_data.apply(lambda row: fill_col(row, 'ext_col') if pd.isna(row['ext_col']) else row['ext_col'], axis=1)
all_data['int_col'] = all_data.apply(lambda row: fill_col(row, 'int_col') if pd.isna(row['int_col']) else row['int_col'], axis=1)

In [ ]:
#因为还有2个ext_col和4个int_col未被填充 直接拿众数填充就行我觉得
all_data['ext_col'].replace(np.nan, all_data['ext_col'].mode()[0], inplace=True)
all_data['int_col'].replace(np.nan, all_data['int_col'].mode()[0], inplace=True)

#### **2.3.5 检查空值并输出**

In [ ]:
#检查一下有没有空值
missing_value = all_data.isnull().sum()
print(missing_value)

In [ ]:
all_data.to_csv('all_data_improved.csv')

In [ ]:
train = all_data[all_data['price'].notna()]  # 价格有值的行
test = all_data[all_data['price'].isna()]    # 价格没有值的行

# 导出到文件时不包含索引
train.to_csv('train_improved.csv', index=False)
test.to_csv('test_improved.csv', index=False)

In [ ]:
train.tail()

### **我觉得这两个缺失的变量可以用其他的回归模型来填充**

## **3. 根据特征数的不同，来选择不同的编码处理**


In [ ]:
pip install category_encoders

In [ ]:
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer

all_data = pd.read_csv('/content/drive/MyDrive/kagglex/all_data_improved.csv')
train = pd.read_csv('/content/drive/MyDrive/kagglex/train_improved.csv')
test = pd.read_csv('/content/drive/MyDrive/kagglex/test_improved.csv')

# # 由于测试集没有价格信息，先添加一个占位列
# test['price'] = -1  # 使用-1作为占位符

# 定义需要目标编码和独热编码的特征
target_encode_cols = ['brand','transmission','ext_col', 'int_col','model']
onehot_encode_cols = ['fuel_type']

# 创建目标编码器，只在训练集上拟合
target_encoder = TargetEncoder(cols=target_encode_cols, smoothing=0.3)
target_encoder.fit(train[target_encode_cols], train['price'])

# 对所有数据进行目标编码
all_data[target_encode_cols] = target_encoder.transform(all_data[target_encode_cols])

# 创建独热编码器，并在所有数据上拟合
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# 创建列转换器，结合使用目标编码和独热编码
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', onehot_encoder, onehot_encode_cols),
    ], remainder='passthrough')

# 拟合所有数据并进行转换，保留原始列
all_data_preprocessed = preprocessor.fit_transform(all_data.drop(columns=['price', 'id']))

# 将预处理后的数据转换回 DataFrame 格式，并添加 'id' 列
all_data_preprocessed_df = pd.DataFrame(all_data_preprocessed, columns=preprocessor.get_feature_names_out())
all_data_preprocessed_df['id'] = all_data['id'].values

# 按照 'id' 列升序排序
all_data_preprocessed_df = all_data_preprocessed_df.sort_values(by='id')

# 检查拆分数据集的维度
X_train_preprocessed_df = all_data_preprocessed_df.iloc[:len(train)].reset_index(drop=True)
X_test_preprocessed_df = all_data_preprocessed_df.iloc[len(train):].reset_index(drop=True)

print("Training set shape:", X_train_preprocessed_df.shape)
print("Test set shape:", X_test_preprocessed_df.shape)


In [ ]:
X_train_preprocessed_df['price'] = train['price']
train = X_train_preprocessed_df
#重命名
new_column_names1 = {
    col: col.split('__')[-1]
    for col in train.columns
}

train = train.rename(columns=new_column_names1)
train.drop(columns=['Unnamed: 0'],inplace=True)
train.tail()

In [ ]:
#X_test_preprocessed_df['price'] = test['price']
test = X_test_preprocessed_df
new_column_names2 = {
    col: col.split('__')[-1]
    for col in test.columns
}
test = test.rename(columns=new_column_names2)
test.drop(columns=['Unnamed: 0'],inplace=True)
test.head()

In [ ]:
#检查一下有没有空值 我打算用回归模型来填充
missing_value = all_data.isnull().sum()
print(missing_value)

In [ ]:
train.to_csv('train_encoded.csv')
test.to_csv('test_encoded.csv')

### **通过下面的相关性来想办法填充上面的空值**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# 计算相关性矩阵
correlation_matrix = train.corr()

# 设置图形大小
plt.figure(figsize=(16, 12))

# 绘制相关性热图
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

# 绘制各个变量与 price 的散点图
# 设置图形大小
plt.figure(figsize=(16, 12))

# # 选择部分变量与 price 绘制散点图，可以选择 top N 个与 price 相关性最大的变量
# top_corr_features = correlation_matrix['price'].abs().sort_values(ascending=False).index[1:11]  # 选择前10个与 price 相关性最大的变量

# for feature in top_corr_features:
#     plt.figure()
#     sns.scatterplot(data=train, x=feature, y='price')
#     plt.title(f'Scatter plot of {feature} vs. price')
#     plt.show()


### **3.1 通过线性回归填充horsepower和engine**

In [ ]:
#再重新合到一起
all = pd.concat([train,test],axis=0,ignore_index=True)
all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314223 entries, 0 to 314222
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   fuel_type_Diesel          314223 non-null  float64
 1   fuel_type_E85 Flex Fuel   314223 non-null  float64
 2   fuel_type_Gasoline        314223 non-null  float64
 3   fuel_type_Hybrid          314223 non-null  float64
 4   fuel_type_Plug-In Hybrid  314223 non-null  float64
 5   fuel_type_not supported   314223 non-null  float64
 6   brand                     314223 non-null  float64
 7   model                     314223 non-null  float64
 8   transmission              314223 non-null  float64
 9   ext_col                   314223 non-null  float64
 10  int_col                   314223 non-null  float64
 11  accident                  314223 non-null  float64
 12  car_age                   314223 non-null  float64
 13  kmilage                   314223 non-null  f

In [ ]:
# 分离数据集
train_horsepower = all[all['horsepower'].notnull()& (all['engine_displacement'].notnull())]
test_horsepower = all[all['horsepower'].isnull()]

train_engine = all[all['engine_displacement'].notnull()& (all['horsepower'].notnull())]
test_engine = all[all['engine_displacement'].isnull()]

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
# 训练预测 horsepower 的模型
features_horsepower = ['fuel_type_Diesel','fuel_type_E85 Flex Fuel','fuel_type_Gasoline','fuel_type_Hybrid',
            'fuel_type_not supported','brand', 'model','transmission','ext_col','int_col','accident','car_age', 'kmilage','engine_displacement']  # 可以根据相关性选择更多特征
features_engine_displacement = ['fuel_type_Diesel','fuel_type_E85 Flex Fuel','fuel_type_Gasoline','fuel_type_Hybrid',
                                'fuel_type_not supported','brand','model','horsepower']

X_train_horsepower = train_horsepower[features_horsepower]
y_train_horsepower = train_horsepower['horsepower']
X_test_horsepower = test_horsepower[features_horsepower]


# 训练模型
model_horsepower = HistGradientBoostingRegressor(
    learning_rate=0.05,
    max_iter=200,
    max_leaf_nodes=31,
    max_depth=None,
    min_samples_leaf=20,
    l2_regularization=0.0,
    max_bins=255,
    random_state=42
)

model_horsepower.fit(X_train_horsepower, y_train_horsepower)

# 预测
test_horsepower['horsepower'] = model_horsepower.predict(X_test_horsepower)

# 填充 horsepower 的空缺值
all.loc[all['horsepower'].isnull(), 'horsepower'] = test_horsepower['horsepower']

# 训练预测 engine_displacement 的模型
X_train_engine = train_engine[features_engine_displacement]
y_train_engine = train_engine['engine_displacement']
X_test_engine = test_engine[features_engine_displacement]

# 训练模型
model_engine = HistGradientBoostingRegressor(
    learning_rate=0.05,
    max_iter=200,
    max_leaf_nodes=31,
    max_depth=None,
    min_samples_leaf=20,
    l2_regularization=0.0,
    max_bins=255,
    random_state=42
)
model_engine.fit(X_train_engine, y_train_engine)

# 预测
test_engine['engine_displacement'] = model_engine.predict(X_test_engine)

# 填充 engine_displacement 的空缺值
all.loc[all['engine_displacement'].isnull(), 'engine_displacement'] = test_engine['engine_displacement']

# 检查是否所有空缺值都已填充
print(all.isnull().sum())

<ipython-input-22-593e5d80dc83>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_horsepower['horsepower'] = model_horsepower.predict(X_test_horsepower)


fuel_type_Diesel                 0
fuel_type_E85 Flex Fuel          0
fuel_type_Gasoline               0
fuel_type_Hybrid                 0
fuel_type_Plug-In Hybrid         0
fuel_type_not supported          0
brand                            0
model                            0
transmission                     0
ext_col                          0
int_col                          0
accident                         0
car_age                          0
kmilage                          0
horsepower                       0
engine_displacement              0
id                               0
price                       125690
dtype: int64


<ipython-input-22-593e5d80dc83>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_engine['engine_displacement'] = model_engine.predict(X_test_engine)


In [ ]:
#这下所有数据齐全了 但我还没做异常值处理 无所谓
all.to_csv('full_data.csv')

In [ ]:
#再拆回来 准备训练！
train = all[all['price'].notnull()]
test = all[all['price'].isnull()]
train.to_csv('train_final.csv')
test.to_csv('test_final.csv')

## **4.开始训练模型**

In [7]:
# 数据读取
train = pd.read_csv('/content/drive/MyDrive/kagglex/train_final.csv')
test = pd.read_csv('/content/drive/MyDrive/kagglex/test_final.csv')

### **4.1 lightgbm**

In [3]:
pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold
from sklearn.metrics import mean_squared_error

def clean_column_names(df):
    df.columns = [
        col.replace(' ', '_')
           .replace(',', '')
           .replace('{', '')
           .replace('}', '')
           .replace('(', '')
           .replace(')', '')
           .replace('[', '')
           .replace(']', '')
           .replace(':', '')
           .replace(';', '')
           .replace("'", '')
           .replace('"', '')
        for col in df.columns
    ]
    return df

# 清理训练和测试数据集的列名
train = clean_column_names(train)
test = clean_column_names(test)

# 拆分训练集
X = train.drop(columns=['price', 'id'])
y = train['price']

# 定义 LGBMRegressor 模型
lgbm = LGBMRegressor()

# 定义随机搜索的参数空间
param_dist = {
    'num_leaves': [20, 31, 40, 50],
    'max_depth': [5, 10, 15, -1],
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'n_estimators': [100, 200, 300, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_samples': [5, 10, 20],
}

# 使用 5 折交叉验证和随机搜索
kf = KFold(n_splits=5, shuffle=True, random_state=42)
random_search = RandomizedSearchCV(
    estimator=lgbm,
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_mean_squared_error',
    cv=kf,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# 执行随机搜索
random_search.fit(X, y)

# 获取最佳参数和模型
best_params = random_search.best_params_
best_model = random_search.best_estimator_
print(f"Best parameters found: {best_params}")

# 对测试集进行预测
X_test = test.drop(columns=['id', 'price'])
predictions = best_model.predict(X_test)

# 将预测结果四舍五入为最近的整数
test['price'] = predictions.round().astype(int)

# 输出形式是一个表格，只保留 test 的 'id' 和 'price' 两列
output = test[['id', 'price']]
output.to_csv('test_predictions_lgbm.csv', index=False)

print("Prediction results saved to 'test_predictions_lgbm.csv'")


Fitting 5 folds for each of 20 candidates, totalling 100 fits


### **4.2 xgboost**

In [8]:
pip install xgboost

In [9]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold
from sklearn.metrics import mean_squared_error

# 拆分训练集
X = train.drop(columns=['price', 'id'])
y = train['price']

# 定义 XGBoost 模型
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# 定义随机搜索的参数空间
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 10,15],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 2, 5, 10,15]
}

# 使用 5 折交叉验证和随机搜索
kf = KFold(n_splits=5, shuffle=True, random_state=42)
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=20,  # 随机搜索的迭代次数
    scoring='neg_mean_squared_error',
    cv=kf,
    verbose=2,
    random_state=42,
    n_jobs=-1  # 使用所有可用的 CPU 核心进行并行计算
)

# 执行随机搜索
random_search.fit(X, y)

# 获取最佳参数
best_params = random_search.best_params_
print(f"Best parameters found: {best_params}")

# 获取 RMSE 分数的详细信息
cv_results = random_search.cv_results_
mean_test_scores = -cv_results['mean_test_score']  # 转为正值，因为得分存储为负的 MSE
std_test_scores = np.sqrt(cv_results['std_test_score'])  # 标准差

# 输出每次迭代的平均 RMSE 和标准差
for mean, std, params in zip(mean_test_scores, std_test_scores, cv_results['params']):
    print(f"RMSE: {np.sqrt(mean):.3f} (+/- {std:.3f}) for {params}")

# 使用最佳参数进行模型训练
best_model = random_search.best_estimator_


# 对测试集进行预测
X_test = test.drop(columns=['id', 'price'])  # 确保 'price' 列不存在于测试集中
predictions = best_model.predict(X_test)

# 将预测结果四舍五入为最近的整数
test['price'] = predictions.round().astype(int)

# 输出形式是一个表格，只保留 test 的 'id' 和 'price' 两列
output = test[['id', 'price']]
output.to_csv('test_predictions_xgboost.csv', index=False)

print("Prediction results saved to 'test_predictions_xgboost.csv'")


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters found: {'subsample': 1.0, 'n_estimators': 300, 'min_child_weight': 5, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
RMSE: 76512.113 (+/- 21031.335) for {'subsample': 1.0, 'n_estimators': 300, 'min_child_weight': 2, 'max_depth': 15, 'learning_rate': 0.1, 'colsample_bytree': 0.6}
RMSE: 73506.481 (+/- 22890.737) for {'subsample': 0.8, 'n_estimators': 200, 'min_child_weight': 15, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
RMSE: 72630.716 (+/- 22912.953) for {'subsample': 1.0, 'n_estimators': 300, 'min_child_weight': 5, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
RMSE: 73564.501 (+/- 22816.974) for {'subsample': 0.8, 'n_estimators': 500, 'min_child_weight': 10, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
RMSE: 80010.036 (+/- 20700.731) for {'subsample': 0.8, 'n_estimators': 500, 'min_child_weight': 10, 'max_depth': 10, 'learning_rate': 0

### **4.3 catboost**

In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split



# 定义特征和目标
X = train.drop(columns=['price','id'])  # 假设 'price' 是目标列
y = train['price']

# 分割训练和验证集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化 CatBoost 回归器
model = CatBoostRegressor(
    iterations=500,             # 最佳迭代次数
    learning_rate=0.01,         # 最佳学习率
    depth=10,                   # 最佳深度
    l2_leaf_reg=5,              # 最佳L2正则化系数
    loss_function='RMSE',
    eval_metric='RMSE',
    random_seed=42,
    verbose=100
)

# 训练模型
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)


# 对测试集进行预测
X_test = test.drop(columns=['id', 'price'])  # 确保 'price' 列不存在于测试集中
test['price'] = model.predict(X_test)

# 输出形式是一个表格，只保留 test 的 'id' 和 'price' 两列
output = test[['id', 'price']]
output.to_csv('test_predictions_xgboost.csv', index=False)
print("Predictions saved to predictions_catboost.csv")


0:	learn: 79735.2398936	test: 74457.4236814	best: 74457.4236814 (0)	total: 157ms	remaining: 1m 18s
100:	learn: 74012.1473134	test: 69122.8004072	best: 69122.8004072 (100)	total: 8.89s	remaining: 35.1s
200:	learn: 72251.5103028	test: 68138.5245875	best: 68138.5245875 (200)	total: 16.1s	remaining: 24s
300:	learn: 71380.6731146	test: 67901.4483336	best: 67901.4483336 (300)	total: 24.7s	remaining: 16.3s
400:	learn: 70803.6825464	test: 67839.6335526	best: 67839.5100305 (397)	total: 31.2s	remaining: 7.71s
499:	learn: 70359.3453707	test: 67828.0378476	best: 67824.6084855 (467)	total: 39.8s	remaining: 0us

bestTest = 67824.60849
bestIteration = 467

Shrink model to first 468 iterations.
Predictions saved to predictions_catboost.csv


In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV, train_test_split


# 定义特征和目标
X = train.drop(columns=['price', 'id'])  # 假设 'price' 和 'id' 是需要移除的列
y = train['price']

# 分割训练和验证集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化 CatBoost 回归器
model = CatBoostRegressor(loss_function='RMSE', random_seed=42, verbose=100)

# 定义参数网格
param_grid = {
    'iterations': [500, 1000],
    'learning_rate': [0.01, 0.03, 0.1],
    'depth': [4, 6, 10],
    'l2_leaf_reg': [1, 3, 5]
}

# 设置 GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)

# 运行 GridSearchCV 找到最佳参数
grid.fit(X_train, y_train)

# 显示最佳参数
print("Best parameters found: ", grid.best_params_)
print("Best RMSE: ", (-grid.best_score_) ** 0.5)

# 使用最佳模型在验证集上进行性能评估
val_predictions = best_model.predict(X_val)
val_rmse = mean_squared_error(y_val, val_predictions, squared=False)
print("Validation RMSE: ", val_rmse)

Fitting 3 folds for each of 54 candidates, totalling 162 fits
0:	learn: 80570.9803043	total: 44ms	remaining: 21.9s
100:	learn: 75775.6250166	total: 3.8s	remaining: 15s
200:	learn: 74497.4837537	total: 6.14s	remaining: 9.14s
300:	learn: 73934.6780317	total: 7.29s	remaining: 4.82s
400:	learn: 73559.1421687	total: 8.43s	remaining: 2.08s
499:	learn: 73250.4931191	total: 9.54s	remaining: 0us
[CV] END depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.01; total time=  10.0s
0:	learn: 78610.4191936	total: 11.9ms	remaining: 5.95s
100:	learn: 73858.6154830	total: 1.21s	remaining: 4.77s
200:	learn: 72667.8333547	total: 2.41s	remaining: 3.59s
300:	learn: 72172.5854460	total: 3.61s	remaining: 2.38s
400:	learn: 71859.5243378	total: 4.75s	remaining: 1.17s
499:	learn: 71612.2721827	total: 7.31s	remaining: 0us
[CV] END depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.01; total time=   7.6s
0:	learn: 80055.3076410	total: 34ms	remaining: 16.9s
100:	learn: 75380.3860200	total: 2.15s	remaining

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


0:	learn: 79735.2398936	total: 63.1ms	remaining: 31.5s
100:	learn: 74012.1473134	total: 6.56s	remaining: 25.9s
200:	learn: 72251.5103028	total: 15s	remaining: 22.3s
300:	learn: 71380.6731146	total: 21.8s	remaining: 14.4s
400:	learn: 70803.6825464	total: 29.8s	remaining: 7.36s
499:	learn: 70359.3453707	total: 36.7s	remaining: 0us
Best parameters found:  {'depth': 10, 'iterations': 500, 'l2_leaf_reg': 5, 'learning_rate': 0.01}
Best RMSE:  73825.90195057672


NameError: name 'best_model' is not defined

In [ ]:
# 使用最佳参数重新训练模型
best_model = grid.best_estimator_

X_test = test.drop(columns=['id', 'price'])  # 确保 'price' 列不存在于测试集中
test['price'] = best_model.predict(X_test)

# 输出形式是一个表格，只保留 test 的 'id' 和 'price' 两列
output = test[['id', 'price']]
output.to_csv('test_predictions_xgboost.csv', index=False)

print("Prediction results saved to 'test_predictions_xgboost.csv'")

### **4.4 Optuna and Lightgbm**

In [10]:
!pip install lightgbm
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00


In [14]:
pip install "dask[dataframe]"  # 或使用conda安装

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 6.9 MB/s eta 0:00:00


In [17]:
def clean_column_names(df):
    df.columns = [
        col.replace(' ', '_')
           .replace(',', '')
           .replace('{', '')
           .replace('}', '')
           .replace('(', '')
           .replace(')', '')
           .replace('[', '')
           .replace(']', '')
           .replace(':', '')
           .replace(';', '')
           .replace("'", '')
           .replace('"', '')
        for col in df.columns
    ]
    return df

# 清理训练和测试数据集的列名
train = clean_column_names(train)
test = clean_column_names(test)

In [20]:
X_train = train.drop(['price', 'id'], axis=1)
y_train = train['price']
X_test = test.drop(['id','price'], axis=1)

In [21]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the objective function for Optuna
def objective(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'regression',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 31, 256),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 100),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'random_state': 42,
        'metric': 'rmse',
        'early_stopping_round': 200  # Singular form for early stopping
    }

    n_splits_ = 3
    skf = KFold(n_splits=n_splits_, shuffle=True, random_state=42)
    cv_scores = []

    # KFold Cross-validation loop
    for fold, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        # Split the preprocessed data into training and validation sets
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]


        # Convert data to LightGBM dataset format
        train_data = lgb.Dataset(X_train_fold, label=y_train_fold)
        val_data = lgb.Dataset(X_val_fold, label=y_val_fold, reference=train_data)

        # Train LightGBM model
        model = lgb.train(
            param,
            train_data,
            valid_sets=[val_data],  # Validation set for early stopping
        )

        # Predict on the validation set
        val_preds = model.predict(X_val_fold, num_iteration=model.best_iteration)

        # Calculate the RMSE for validation set predictions
        score = mean_squared_error(y_val_fold, val_preds, squared=False)
        cv_scores.append(score)

    # Return the mean RMSE across all folds (negated because Optuna minimizes the objective)
    return np.mean(cv_scores)

# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Optimize the study (you can increase n_trials for a more extensive search)
study.optimize(objective, n_trials=30)

# Get the best hyperparameters
best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

# Train the final model with the best hyperparameters
best_params['metric'] = 'rmse'  # Add back the metric parameter
final_model = lgb.train(
    best_params,
    lgb.Dataset(X_train, label=y_train),
    valid_sets=[lgb.Dataset(X_train, label=y_train)]
)

# Predict on the test set
y_test_pred = final_model.predict(X_test)

# Create submission dataframe
submission_df = pd.DataFrame({
    'id': test['id'].values,
    'price': y_test_pred
})

# Save the submission file to CSV
submission_df.to_csv('submission_optuna_lgbm.csv', index=False)

[I 2024-09-28 15:19:09,418] A new study created in memory with name: no-name-03998a39-90e1-4232-9411-d15e69f9a2b5
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020171 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1629
[LightGBM] [Info] Number of data points in the train set: 125688, number of used features: 16
[LightGBM] [Info] Start training from score 43995.557643
Training until validation scores don't improve for 200 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026337 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1632
[LightGBM] [Info] Number of data points in the train set: 125689, number of used features: 16
[LightGBM] [Info] Start training from score 43873.216566
Training until validation scores don't improve for 200 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010031 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1632
[LightGBM] [Info] Number of data points in the train set: 125689, number of used features: 16
[LightGBM] [Info] Start training from score 43873.216566
Training until validation scores don't improve for 200 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007499 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1631
[LightGBM] [Info] Number of data points in the train set: 125689, number of used features: 16
[LightGBM] [Info] Start training from score 43765.275259
Training until validation scores don't improve for 200 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-09-28 15:35:43,063] Trial 29 finished with value: 72474.97355451305 and parameters: {'n_estimators': 810, 'learning_rate': 0.024523428216820832, 'num_leaves': 163, 'max_depth': 16, 'min_child_weight': 78, 'lambda_l1': 7.056655304946356, 'lambda_l2': 6.133016693121997, 'subsample': 0.8617726060593139, 'colsample_bytree': 0.5919510608625638}. Best is trial 15 with value: 72398.62454417242.
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Best Hyperparameters: {'n_estimators': 853, 'learning_rate': 0.016628274229682542, 'num_leaves': 130, 'max_depth': 8, 'min_child_weight': 88, 'lambda_l1': 4.905644138434035, 'lambda_l2': 7.246770057896177, 'subsample': 0.8261359410344453, 'colsample_bytree': 0.5770956040643702}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032128 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1631
[LightGBM] [Info] Number of data points in the train set: 188533, number of used features: 16
[LightGBM] [Info] Start training from score 43878.016178
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf